In [ ]:
!pip install import-ipynb

In [ ]:
import import_ipynb
import LoadingDataset  as dataset_loader
import ModuleArchitecture as md_architecture

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
! pwd
%cd drive/MyDrive
%cd Colab-Super_resolution/

#Loading and visualize dataset

#Building and Training Model